Я решаю задания варинта 3, но задачи актуальны и для остальных.

###Задание 1

In [ ]:
import numpy as np
array = np.array([63.4, 64.5, 57.1, 51.7, 40.1, 37.7, 45.8, 54.9, 35.9, 31.0, 35.5, 19.2, 13.6, 31.4, 40.1])
array.mean()

41.46000000000001

In [ ]:
summa = 0
for i in range(len(array)):
  summa+=array[i]**2
summa/=len(array)
D = summa-array.mean()**2
D

208.5143999999991

In [ ]:
s2 = len(array)/(len(array)-1) * D
s2

223.40828571428474

In [ ]:
s = s2**0.5
s

14.946848688412041

Дальше находим по таблице квантиль t_{(gamma+1)/2}

Вам нужно поменять тут значение 

In [ ]:
t = 1.761

X с чертой - t*(n-1)*s/sqrt(n) < m < X с чертой + t*(n-1)*s/sqrt(n)

In [ ]:
left = array.mean() - t*s/(len(array)**0.5)
left

34.663843603905875

In [ ]:
right = array.mean() + t*s/(len(array)**0.5)
right

48.25615639609414

###Задание 2

In [ ]:
n = 100
mean_x = 82 
sum_x2 = 686800
lamb = 0.98

В данном случае объем выборки велик, используем формулу: 

X с чертой - t * sigma/(sqrt(n)) < m < X с чертой + t * sigma/(sqrt(n))

In [ ]:
sigma = (sum_x2/n - mean_x**2)**0.5
sigma

12.0

In [ ]:
F = (1 + lamb)/2
F

0.99

In [ ]:
t = 2.33

In [ ]:
left = mean_x - t*sigma/(n**0.5)
left

79.204

In [ ]:
right = mean_x + t*sigma/(n**0.5)
right

84.796

###Задание 3

Оценка lambda -- lambda с крышкой равна X с чертой. 

In [258]:
import pandas as pd
alpha = 0.01
df = pd.DataFrame(columns=["Amount of deaths", "Number of investigations"])
deaths = [0, 1, 2, 3, 4]
investigations = [109, 65, 22, 3, 1]
for i in range(len(deaths)):
  df.loc[len(df.index)] = [deaths[i], investigations[i]]
df

,Amount of deaths,Number of investigations
0,0,109
1,1,65
2,2,22
3,3,3
4,4,1


In [259]:
h_eval = 0;
for i in range(len(deaths)):
  h_eval+=deaths[i]*investigations[i]
h_eval/=(df["Number of investigations"].sum())
h_eval

0.61

In [260]:
import numpy as np
import math
df = pd.DataFrame(columns=["Amount of deaths", "Number of investigations", "Probability", "Theoretical frequencies"])
n = sum(investigations)
for i in range(len(deaths)):
  amount_of_deaths = deaths[i]
  number_of_investigations = investigations[i]
  probability = (((h_eval)**amount_of_deaths)/(np.math.factorial(amount_of_deaths))) * math.e**(-h_eval)
  theoretical_frequencies = round(n * probability)
  df.loc[len(df.index)] = [amount_of_deaths, number_of_investigations, probability, theoretical_frequencies]

df

,Amount of deaths,Number of investigations,Probability,Theoretical frequencies
0,0.0,109.0,0.543351,109.0
1,1.0,65.0,0.331444,66.0
2,2.0,22.0,0.101090,20.0
3,3.0,3.0,0.020555,4.0
4,4.0,1.0,0.003135,1.0


Выведем сумму ni

In [261]:
df["Number of investigations"].sum()

200.0

Выведем сумму ni*

In [262]:
df["Theoretical frequencies"].sum()

200.0

In [263]:
array_of_bad_ones = [0, 0]
pi = df["Probability"]
ni = df["Number of investigations"]

df = pd.DataFrame(columns=["ni", "ni*"])

for i in range(len(pi)):
  if(n*pi[i]>=5):
    table_ni = ni[i]
    table_ni_star = round(n*pi[i])
    df.loc[len(df.index)] = [table_ni, table_ni_star]
  else:
    table_ni = ni[i]
    table_ni_star = round(n*pi[i])
    array_of_bad_ones[0]+=table_ni
    array_of_bad_ones[1]+=table_ni_star

if(array_of_bad_ones[1]<5):
  df.loc[len(df.index)-1][0] += array_of_bad_ones[0]
  df.loc[len(df.index)-1][1] += array_of_bad_ones[1]
else:
  df.loc[len(df.index)] = array_of_bad_ones
ni = df["ni"]
ni_star = df["ni*"]

df = pd.DataFrame(columns=["ni", "ni*", "(ni-ni*)^2 / ni*"])
for i in range(len(ni)):
  table_ni = ni[i]
  table_ni_star = ni_star[i]
  table_X2 = (table_ni - table_ni_star)**2 / (table_ni_star)
  df.loc[len(df.index)] = [table_ni, table_ni_star, table_X2]
df


,ni,ni*,(ni-ni*)^2 / ni*
0,109.0,109.0,0.000000
1,65.0,66.0,0.015152
2,22.0,20.0,0.200000
3,4.0,5.0,0.200000


In [264]:
resulting_table = pd.DataFrame(columns=["ni", "ni*", "X2_invest" ])

resulting_table.loc[len(resulting_table.index)] = [df["ni"].sum(), df["ni*"].sum(), df["(ni-ni*)^2 / ni*"].sum()]
resulting_table

,ni,ni*,X2_invest
0,200.0,200.0,0.415152


Теперь вам осталось дишь сравнить ответ с таблицей: 
По заданному уровню значимости a = 0.01 и таблице квантилей распределения X2 находим квартиль порядка 1 - a с k -l - 1 степенью свободы и определяем X2_крит = X2_{1-a}(k - l - 1). Параметр k равен числу групп после объединения малочисленных групп: k = 4, параметр l равен числу неизвестных параметров, от которых зависит распределение: для Пуассона l=1.  

в моем случае: 0.415152 < 9,21 => гипотеза подтвердилась!